In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

# 导入tensorboard
%load_ext tensorboard

time: 8.68 ms (started: 2021-08-13 03:39:57 +08:00)


In [2]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 #1 Aug 13, 2021'

git push origin master

origin	git@github.com:ustchope/kerastuner.git (fetch)
origin	git@github.com:ustchope/kerastuner.git (push)
[master 9a5e7ff] 更新 #1 Aug 13, 2021
 2 files changed, 132 insertions(+), 31 deletions(-)
 create mode 100644 "\345\217\257\350\247\206\345\214\226\350\266\205\345\217\202\346\225\260\350\260\203\346\225\264\350\277\207\347\250\213.ipynb"


To github.com:ustchope/kerastuner.git
   2a9432b..9a5e7ff  master -> master


time: 3.69 s (started: 2021-08-13 02:39:20 +08:00)


In [3]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 1.47 s (started: 2021-08-13 02:39:38 +08:00)


# 介绍

KerasTuner 将日志打印到屏幕上，包括每次试验中的超参数值，供用户监控进度。 然而，阅读日志不够直观，无法感知超参数对结果的影响，因此，我们提供了一种使用 TensorBaord 通过交互式图形可视化超参数值和相应评估结果的方法。

TensorBoard 是用于可视化机器学习实验的有用工具。 它可以在模型训练期间监控损失和指标，并可视化模型架构。 使用 TensorBoard 运行 KerasTuner 将为您提供使用其 HParams 插件可视化超参数调整结果的附加功能。

我们将使用一个为 MNIST 图像分类数据集调整模型的简单示例来展示如何将 KerasTuner 与 TensorBoard 结合使用。

第一步是下载和格式化数据。

In [4]:
import numpy as np
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# Normalize the pixel values to the range of [0, 1].
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Add the channel dimension to the images.
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
# Print the shapes of the data.
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)
time: 767 ms (started: 2021-08-13 02:46:38 +08:00)


然后，我们编写一个 `build_model` 函数来使用超参数构建模型并返回模型。 超参数包括要使用的模型类型（多层感知器或卷积神经网络）、层数、单元或过滤器的数量、是否使用 dropout。

In [5]:
def build_model(hp):
    inputs = keras.Input(shape=(28, 28, 1))
    # Model type can be MLP or CNN.
    model_type = hp.Choice("model_type", ["mlp", "cnn"])
    x = inputs
    if model_type == "mlp":
        x = layers.Flatten()(x)
        # Number of layers of the MLP is a hyperparameter.
        for i in range(hp.Int("mlp_layers", 1, 3)):
            # Number of units of each layer are
            # different hyperparameters with different names.
            output_node = layers.Dense(
                units=hp.Int(f"units_{i}", 32, 128, step=32), activation="relu",
            )(x)
    else:
        # Number of layers of the CNN is also a hyperparameter.
        for i in range(hp.Int("cnn_layers", 1, 3)):
            x = layers.Conv2D(
                hp.Int(f"filters_{i}", 32, 128, step=32),
                kernel_size=(3, 3),
                activation="relu",
            )(x)
            x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        x = layers.Flatten()(x)

    # A hyperparamter for whether to use dropout layer.
    if hp.Boolean("dropout"):
        x = layers.Dropout(0.5)(x)

    # The last layer contains 10 units,
    # which is the same as the number of classes.
    outputs = layers.Dense(units=10, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model.
    model.compile(
        loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam",
    )
    return model

time: 1.95 ms (started: 2021-08-13 02:49:52 +08:00)


我们可以对模型进行快速测试，以检查它是否为 CNN 和 MLP 成功构建。

In [6]:
# Initialize the `HyperParameters` and set the values.
hp = kt.HyperParameters()
hp.values["model_type"] = "cnn"
# Build the model using the `HyperParameters`.
model = build_model(hp)
# Test if the model runs with our data.
model(x_train[:100])
# Print a summary of the model.
model.summary()

# Do the same for MLP model.
hp.values["model_type"] = "mlp"
model = build_model(hp)
model(x_train[:100])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shap

使用 10 次试验初始化 RandomSearch 调谐器，并使用验证准确性作为选择模型的指标。

In [7]:
tuner = kt.RandomSearch(
    build_model,
    max_trials=10,
    # Do not resume the previous search in the same directory.
    overwrite=True,
    objective="val_accuracy",
    # Set a directory to store the intermediate results.
    directory="/tmp/tb",
)

time: 144 ms (started: 2021-08-13 02:55:37 +08:00)


通过调用 `tuner.search(...)` 开始搜索。 要使用 TensorBoard，我们需要将 `keras.callbacks.TensorBoard` 实例传递给回调。

In [8]:
tuner.search(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=2,
    # Use the TensorBoard callback.
    # The logs will be write to "/tmp/tb_logs".
    callbacks=[keras.callbacks.TensorBoard("/tmp/tb_logs")],
)

Trial 10 Complete [00h 00m 22s]
val_accuracy: 0.9205833077430725

Best val_accuracy So Far: 0.9838333129882812
Total elapsed time: 00h 03m 47s
INFO:tensorflow:Oracle triggered exit
time: 3min 47s (started: 2021-08-13 02:59:02 +08:00)


In [ ]:
如果在 Colab 中运行，以下两个命令将显示 Colab 中的 TensorBoard。

In [4]:
%tensorboard --logdir /tmp/tb_logs --port 6007

Reusing TensorBoard on port 6007 (pid 942281), started 0:00:13 ago. (Use '!kill 942281' to kill it.)

time: 8.33 ms (started: 2021-08-13 03:41:46 +08:00)


您可以访问 TensorBoard 的所有常见功能。 例如，您可以查看损失和指标曲线，并可视化不同试验中模型的计算图。

除了这些功能之外，我们还有一个 HParams 选项卡，其中包含三个视图。 在表格视图中，您可以查看具有不同超参数值和评估指标的表格中的 10 个不同试验。

在左侧，您可以为某些超参数指定过滤器。 例如，您可以指定仅查看没有 dropout 层和 1 到 2 个密集层的 MLP 模型。

除了表格视图，它还提供了另外两个视图，平行坐标视图和散点图矩阵视图。 它们只是相同数据的不同可视化方法。 您仍然可以使用左侧的面板来过滤结果。

在平行坐标视图中，每条彩色线都是一个试验。 轴是超参数和评估指标。

在散点图矩阵视图中，每个点都是一个试验。 这些图是试验在以不同超参数和度量为轴的平面上的投影。